In [18]:
import torch
import torchvision
import scipy.misc

We will use MNIST datasets to learn basic of MLP neural network. They contain image samples of written numbers. It perfectly suits study over simple neural networks.

In [3]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Here we will load sample of our data to see if we can recognize number based on given arrays. 

In [22]:
train_image, train_target = trainset[2]    #let us examine the 0-th sample
train_image

Arrays are too long to fit in one row, therefore it is harder for human to classify numbers correctrly. If we change index numer we will see array for another image. Valuse are in range 1-256, becasue MNIST dataset uses RGB colour pallet. 

In [8]:
trainset.data[2]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,  67, 232,  39,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,  62,  81,   0,   

In [9]:
train_target

4

To receive images in greyscale we have to normalize it. We can achive it by obtaining mean and standard deviation. After that we have to divide it by range of valuse (in case of RBG pallet it is 255).

In [24]:
(trainset.data.numpy().mean()/255.0, trainset.data.numpy().std()/255.0)

(0.1306604762738429, 0.30810780385646264)

Below we create sets on wich our neural network weill learn. First we have to  standarize it and convert to torch float tensor form.

In [25]:
transform = torchvision.transforms.Compose(
    [ torchvision.transforms.ToTensor(), #Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
      torchvision.transforms.Normalize((0.1307), (0.3081))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=2048, shuffle=False)   #we do shuffle it to give more randomizations to training epochs

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

Below we visualize our training labels for first 5 batches.

In [29]:
for i, data in enumerate(trainloader):
        batch_inputs, batch_labels = data

        if i<5:
            print(i, "-th batch labels :", batch_labels)

0 -th batch labels : tensor([5, 0, 4,  ..., 1, 4, 1])
1 -th batch labels : tensor([7, 5, 4,  ..., 9, 3, 9])
2 -th batch labels : tensor([2, 4, 9,  ..., 7, 1, 2])
3 -th batch labels : tensor([2, 9, 0,  ..., 1, 5, 6])
4 -th batch labels : tensor([3, 0, 1,  ..., 0, 3, 1])


Below we check number of order in batch_inputs which will be used in our neural network. Here we can see how our two-dimensional image changed to four dimensions. First determins batch size, second is index in dataset, third and fourth are size of our image.

In [48]:
for i, data in enumerate(trainloader):
        batch_inputs, batch_labels = data

        if i==0:
            print(i, "-th batch inputs :", batch_inputs.size())

0 -th batch inputs : torch.Size([2048, 1, 28, 28])


Finally we can make our first MLP neural network. To do so we create class using torch.nn.Module. We can create many layers. There is no universal rule on how many layers or perceptrons our network should have. To get the best results we have to experiment with few different setup. Good tip is to make our network grow first, then with each layer lower number of perceptrons. **Proceed with caution. If we give too much information to our network it will overtrain and give bad results!**

In [60]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Flatten(),   #change the last three orders in data (with dimensions 1, 28 and 28 respectively) into one order of dimensions (1*28*28)
            torch.nn.Linear(1*28*28, 512),
            torch.nn.Sigmoid(),
            torch.nn.Linear(512, 1024), #remember that entry size (first index) of our layer must be the same as size of previous layer (second index)
            torch.nn.Sigmoid(),
            torch.nn.Linear(1024, 2048),
            torch.nn.Sigmoid(),
            torch.nn.Linear(2048, 512),
            torch.nn.Sigmoid(),
            torch.nn.Linear(512,10)
        )
    def forward(self, x):
        out = self.mlp(x)
        return out

Below we use our neural network. We create object of our MLP class. 

In [61]:
net = MLP()
optimizer = torch.optim.Adam(net.parameters(), 0.001)   #initial and fixed learning rate of 0.001

net.train() #here we inform our network it will be training in next lines
for epoch in range(8):  #  an epoch is a training run through the whole data set

    loss = 0.0
    for batch, data in enumerate(trainloader):
        batch_inputs, batch_labels = data
        #batch_inputs.squeeze(1)     #alternatively if not for a Flatten layer, squeeze() could be used to remove 
                                      #the second order of the tensor, the Channel, which is one-dimensional (this index can be equal to 0 only)
        
        optimizer.zero_grad()

        batch_outputs = net(batch_inputs)
        loss = torch.nn.functional.cross_entropy(batch_outputs, batch_labels, reduction = "mean")
        print("epoch:", epoch, "batch:", batch, "current batch loss:", loss.item()) 
        loss.backward()       #this computes gradients with backward step. 
        optimizer.step()     #but this line in fact updates our neural network. 
                                ####You can experiment - comment this line and check, that the loss DOES NOT improve, meaning that the network doesn't update

epoch: 0 batch: 0 current batch loss: 2.3200719356536865
epoch: 0 batch: 1 current batch loss: 2.6219561100006104
epoch: 0 batch: 2 current batch loss: 2.428776264190674
epoch: 0 batch: 3 current batch loss: 2.373007297515869
epoch: 0 batch: 4 current batch loss: 2.348644256591797
epoch: 0 batch: 5 current batch loss: 2.3615713119506836
epoch: 0 batch: 6 current batch loss: 2.357271432876587
epoch: 0 batch: 7 current batch loss: 2.3340818881988525
epoch: 0 batch: 8 current batch loss: 2.3091094493865967
epoch: 0 batch: 9 current batch loss: 2.3047876358032227
epoch: 0 batch: 10 current batch loss: 2.304500102996826
epoch: 0 batch: 11 current batch loss: 2.308112144470215
epoch: 0 batch: 12 current batch loss: 2.3073275089263916
epoch: 0 batch: 13 current batch loss: 2.306042432785034
epoch: 0 batch: 14 current batch loss: 2.3049612045288086
epoch: 0 batch: 15 current batch loss: 2.3078527450561523
epoch: 0 batch: 16 current batch loss: 2.304643392562866
epoch: 0 batch: 17 current batch

In next cell we add scheduler. It will update learning rate after each epoch. Usually it improves network giving information on learning rate, which can show us if we didn't overtain our neural network as well.

In [62]:
net_with_scheduler = MLP()
optimizer = torch.optim.Adam(net_with_scheduler.parameters(), 0.001)   #initial learning rate of 0.001. 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)    #updates the learning rate after each epoch. There are many ways to do that: 
                                                                                  #StepLR multiplies learning rate by gamma

net_with_scheduler.train()
for epoch in range(9):  #  an epoch is a training run through the whole data set

    loss = 0.0
    for batch, data in enumerate(trainloader):
        batch_inputs, batch_labels = data
        #batch_inputs.squeeze(1)     #alternatively if not for a Flatten layer, squeeze() could be used to remove the second order of the tensor, 
                                      #the Channel, which is one-dimensional (this index can be equal to 0 only)
        
        optimizer.zero_grad()

        batch_outputs = net_with_scheduler(batch_inputs)
        loss = torch.nn.functional.cross_entropy(batch_outputs, batch_labels, reduction = "mean")
        print("epoch:", epoch, "batch:", batch, "current batch loss:", loss.item(), "current lr:", scheduler.get_last_lr()[0]) 
        loss.backward()       #this computes gradients as we have seen in previous workshops
        optimizer.step()     #but this line in fact updates our neural network. 
                                
    scheduler.step()

epoch: 0 batch: 0 current batch loss: 2.3618810176849365 current lr: 0.001
epoch: 0 batch: 1 current batch loss: 2.5173678398132324 current lr: 0.001
epoch: 0 batch: 2 current batch loss: 2.4072515964508057 current lr: 0.001
epoch: 0 batch: 3 current batch loss: 2.3458774089813232 current lr: 0.001
epoch: 0 batch: 4 current batch loss: 2.320033073425293 current lr: 0.001
epoch: 0 batch: 5 current batch loss: 2.3313136100769043 current lr: 0.001
epoch: 0 batch: 6 current batch loss: 2.3503170013427734 current lr: 0.001
epoch: 0 batch: 7 current batch loss: 2.341620445251465 current lr: 0.001
epoch: 0 batch: 8 current batch loss: 2.3136448860168457 current lr: 0.001
epoch: 0 batch: 9 current batch loss: 2.302180767059326 current lr: 0.001
epoch: 0 batch: 10 current batch loss: 2.298361301422119 current lr: 0.001
epoch: 0 batch: 11 current batch loss: 2.2976691722869873 current lr: 0.001
epoch: 0 batch: 12 current batch loss: 2.2990074157714844 current lr: 0.001
epoch: 0 batch: 13 current

We have built our networks. No we can see how accurate they actually are. To do that we will use simple method. Check how many accurate results we got and divide it by all samples. We will check how it went without and with sheduler.

In [64]:
good = 0
wrong = 0

net.eval()   #it prevents that the net learns during evalution. The gradients are not computed, so this makes it faster, too
#batches in test are of size 1
for batch, data in enumerate(testloader):
    datapoint, label = data
    
    prediction = net(datapoint)                  #prediction has values representing the "prevalence" of the corresponding class
    classification = torch.argmax(prediction)    #the class is the index of maximal "prevalence"
    
    if classification.item() == label.item():
        good += 1
    else:
        wrong += 1
        
print("accuracy = ", good/(good+wrong))

accuracy =  0.9615


In [65]:
good = 0
wrong = 0

net_with_scheduler.eval()   #it prevents that the net learns during evalution. The gradients are not computed, so this makes it faster, too
#batches in test are of size 1
for batch, data in enumerate(testloader):
    datapoint, label = data
    
    prediction = net_with_scheduler(datapoint)                  #prediction has values representing the "prevalence" of the corresponding class
    classification = torch.argmax(prediction)                   #the class is the index of maximal "prevalence"
    
    if classification.item() == label.item():
        good += 1
    else:
        wrong += 1
        
print("accuracy = ", good/(good+wrong))

accuracy =  0.9508


Results are quite good. Both methods got over 95% accuracy. To improve it even more we can change number of perceptrons, sheduler properities and activation method.